In [ ]:
import cupy as cp
import requests
class API:
    base = "https://recruitment_test.investcloud.com/api/numbers"
    cols = {
        "A": [],
        "B": []
    }
    rows = {
        "A": [],
        "B": []
    }
    
    def __init__(self, size):
        self.size = size
        requests.get(f"{self.base}/init/{size}")
    
    def getCols(self, matrix, col):
        assert matrix == "A" or matrix == "B"
        assert col >= 0 and col < self.size
        return requests.get(f"{self.base}/{matrix}")
    
    def getRows(self, matrix, row):
        assert matrix == "A" or matrix == "B"
        assert row >= 0 and row < self.size
        return requests.get(f"{self.base}/{matrix}/{row}")